In [2]:
import glob
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.express as px

Going to start by creating dataframes for given league with *all* stats in a single 

In [14]:
df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[0]}.csv")
# df_all = []# Empty list which we'll be adding dfs to
# for league in leagues:
#     df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('', f"../data/raw/team/{league}*.csv"))), axis=1).reset_index(drop=True)
#     df = pd.concat()
# df_all = pd.concat(df_all, axis=0, ignore_index=True)


In [4]:
# Function to normalize to per-90 stats (i.e. per full game equivalent)
df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[0]}.csv")
def shooting_preprocess(df, scale):
    '''Preprocess 'Shooting' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns
        - df: processed dataframe'''
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Drop dead-ball opportunities
    df.drop(columns=['FK', 'PK', 'PKatt', 'xG', 'G-xG'], inplace=True) 

    # Drop columns that are already per 90
    df.drop(columns=['Sh', 'SoT'], inplace=True) 

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)

    # executing the function
    df[["Gls", "npxG", "np:G-xG"]] = df[["Gls", "npxG", "np:G-xG"]].apply(per_90s, games)

    # Rename to reflect 'per-90' status
    df.rename(columns={"Gls": "Gls/90", "npxG": "npxG/90"}, inplace=True)

    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)

    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)

    return df

In [5]:
df.columns

Index(['Squad', '90s', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh',
       'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG', 'npxG/Sh', 'G-xG',
       'np:G-xG'],
      dtype='object')

In [6]:

df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[1]}.csv")

def passing_preprocess(df, scale):
    '''Preprocess 'Passing' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns
        - df: processed dataframe'''
    
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    # executing the per-90 function on relevant stats.
    df[["Cmp", "Att", "TotDist", "PrgDist", "Cmp.1", "Att.1", "Cmp.2", "Att.2", "Cmp.3", "Att.3",
        "Ast", "xA", "KP", "1/3", "PPA", "CrsPA", "Prog" ]] = df[["Cmp", "Att", "TotDist", "PrgDist", "Cmp.1", 
        "Att.1", "Cmp.2", "Att.2", "Cmp.3", "Att.3",
        "Ast", "xA", "KP", "1/3", "PPA", "CrsPA", "Prog" ]].apply(lambda x: x/games)
    
    # Normalize distances so they are 'distances per pass'
    df["TotDist"]= df["TotDist"]/df["Att"]
    df.rename(columns={"TotDist": "TotDist/pass"}, inplace=True)

    df["PrgDist"] = df["PrgDist"]/df["Prog"]
    df.rename(columns={"PrgDist": "Dist/PrgPass"}, inplace=True)

    # Rename short/medium/long
    df.rename({"Att.1": "ShortAtt", "Cmp%.1": "Short%", "Att.2": "MedAtt", "Cmp%.2": "Med%","Att.3": "LongAtt", "Cmp%.3": "Long%"})


    # Find proportion of total passes that are short ( <5 yds), medium (5-15 yds) or long (> 15 yds)
    df["PropShort"] = df["ShortAtt"]/df["Att"]
    
    df["PropMed"] = df["MedAtt"]/df["Att"]
    
    df["PropLong"]= df["LongAtt"]/df["Att"]



    # As above, but with other passing stats
    df["KP/Pass"] = df["KP"]/df["Cmp"]
    
    df["FinalThird/Pass"]= df["1/3"]/df["Cmp"]
    
    df["PPA/Pass"]= df["PPA"]/df["Cmp"]
    
    df["CrsPA/Pass"]= df["CrsPA"]/df["Cmp"]

    # Drop columns we won't need anymore
    df.drop(columns=['Cmp', 'Att', 'Cmp%', 'Cmp.1', 'Cmp.2', 'Cmp.3', 'KP', '1/3', 'PPA', 'CrsPA','Prog'], inplace=True) 
    
    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)
    
    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)
    
    return df 

In [7]:
new_df = passing_preprocess(df)

In [8]:
new_df

,Squad,TotDist/pass,Dist/PrgPass,Att.1,Cmp%.1,Att.2,Cmp%.2,Att.3,Cmp%.3,Ast,xA,A-xA,PropShort,PropMed,PropLong,KP/Pass,FinalThird/Pass,PPA/Pass,CrsPA/Pass
0,Alavés,15.348728,106.947557,121.763158,80.1,134.500000,78.3,100.736842,51.8,0.526316,0.597368,-2.7,0.327761,0.362046,0.271162,0.025102,0.081707,0.020528,0.008130
1,Athletic Club,14.900729,71.976969,163.868421,85.4,178.526316,82.6,93.921053,54.9,0.894737,0.942105,-1.8,0.360234,0.392456,0.206468,0.025852,0.079760,0.023799,0.006767
2,Atlético Madrid,15.131974,77.632800,198.605263,89.0,184.131579,85.9,100.605263,57.2,1.184211,1.000000,7.0,0.393524,0.364845,0.199343,0.022420,0.072358,0.021962,0.004902
3,Barcelona,16.377071,71.527262,261.263158,92.0,267.736842,90.5,107.578947,71.9,1.368421,1.168421,7.6,0.397279,0.407123,0.163585,0.018383,0.079858,0.018891,0.004203
4,Betis,15.737054,80.779388,194.131579,89.5,195.421053,87.6,99.000000,61.5,1.184211,0.997368,7.1,0.380866,0.383396,0.194228,0.024888,0.070673,0.019821,0.003166
5,Cádiz,15.332591,95.492398,123.000000,84.2,135.342105,81.2,104.210526,51.1,0.552632,0.671053,-4.5,0.325080,0.357699,0.275421,0.026448,0.077020,0.022864,0.010366
6,Celta Vigo,15.662374,84.355172,196.657895,87.3,205.236842,85.5,114.552632,60.4,0.763158,0.736842,1.0,0.364501,0.380402,0.212321,0.016924,0.063928,0.018653,0.005497
7,Elche,15.837399,101.786325,155.105263,87.3,167.921053,86.0,101.157895,57.5,0.763158,0.736842,1.0,0.350666,0.379641,0.228701,0.019536,0.062729,0.016331,0.006029
8,Espanyol,15.477451,95.711991,163.552632,89.5,170.815789,87.2,90.842105,56.2,0.657895,0.686842,-1.1,0.369764,0.386185,0.205378,0.021953,0.071588,0.016297,0.004763
9,Getafe,15.018306,101.705036,131.736842,83.6,133.789474,79.9,99.973684,51.4,0.684211,0.518421,6.3,0.344505,0.349873,0.261441,0.027022,0.072729,0.016577,0.006037


In [36]:
df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[2]}.csv")
print(df.columns)

def pass_types_preprocess(df, scale):
    '''Preprocess 'Pass Type' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns
        - df: processed dataframe'''
    
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    # Some stats here are redundant from the "Passing" stat, so we'll drop them
    # Also some we just don't care about: dead balls, "Other" passes (which is likely keeper throwing it)
    df.drop(columns = ['FK', 'CK', 'In', 'Out', 'Str', 'Other'], inplace=True)

    # Offsides could be interesting, as it suggests through/attacking balls, but it is only like 2% of passes, so let's drop
    df.drop(columns = ['Off'], inplace=True)


    # executing the per-90 function on relevant stats
    df[["Att", "Live", "Dead", "TB", "Press", "Sw", "Crs", "Ground", "Low", "High", "Left", "Right", "Head", "TI", 
        "Cmp", "Out.1", "Int", "Blocks" ]] =  df[["Att", "Live", "Dead", "TB", "Press", "Sw", "Crs", "Ground", "Low", "High", 
        "Left", "Right", "Head", "TI", "Cmp", "Out.1", "Int", "Blocks" ]].apply(lambda x: x/games)
    
    # Normalize certain stats to be per pass
    # 'TB' (completed pass b/w back two defenders into open space) has a max of 0.63-per-90, so don't normalize
    live_passes = df['Live'][0]
    df[["Press", "Sw", "Crs", "Out.1", "Int", "Blocks"]] = df[[ "Press", "Sw", "Crs", "Out.1", "Int", "Blocks"]].apply(lambda x: x/live_passes)
    df.rename(columns ={"Out.1": "PropOut", "Int": "PropInt", "Blocks": "PropBlocks"}, inplace=True)
    
    # For "Height", it includes dead-ball kicks (which we don't care about). We'll do a sill trick:
    # Assume 10% of Dead ball kicks are 'low', 10% 'ground' and 80% are 'high'. Then subtract those amounts from the height stats
    df["Ground"] = df["Ground"] - 0.1*df["Dead"]
    df["Low"] = df["Low"] - 0.1*df["Dead"]
    df["High"] = df["High"] - 0.8*df["Dead"]

    # Then, find the proportion of each height...
    df["Ground"] = df["Ground"]/df["Live"]
    df["Low"] = df["Low"]/df["Live"]
    df["High"] = df["High"]/df["Live"]
    df.rename(columns={"Ground": "PropGround", "Low": "PropLow", "High": "PropHigh"}, inplace=True)

     #...and the proportion from each body part (no normalization from dead balls though)
    df["Left"] = df["Left"]/df["Live"]
    df["Right"] = df["Right"]/df["Live"]
    df["Head"] = df["Head"]/df["Live"]
    df.rename(columns={"Left": "PropLeft", "Right": "PropRight", "Head": "PropHead"}, inplace=True)

    # Drop columns we won't need anymore
    df.drop(columns=['Att', 'Dead'], inplace=True) 
    
    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)

    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)
    
    return df 

Index(['Unnamed: 0', 'Squad', '# Pl', '90s', 'Att', 'Live', 'Dead', 'FK', 'TB',
       'Press', 'Sw', 'Crs', 'CK', 'In', 'Out', 'Str', 'Ground', 'Low', 'High',
       'Left', 'Right', 'Head', 'TI', 'Other', 'Cmp', 'Off', 'Out.1', 'Int',
       'Blocks'],
      dtype='object')


In [37]:
new_df = pass_types_preprocess(df)
new_df

,Squad,Live,TB,Press,Sw,Crs,PropGround,PropLow,PropHigh,PropLeft,PropRight,PropHead,TI,Cmp,PropOut,PropInt,PropBlocks
0,Alavés,319.605263,0.236842,0.202882,0.049650,0.040922,0.577752,0.165319,0.256929,0.292878,0.623878,0.085961,21.736842,258.947368,0.032771,0.034500,0.036147
1,Athletic Club,402.026316,0.657895,0.206752,0.045945,0.043804,0.690194,0.147810,0.161995,0.259933,0.703410,0.056097,23.131579,346.105263,0.035488,0.047674,0.037382
2,Atlético Madrid,453.710526,0.868421,0.221408,0.052367,0.039934,0.717261,0.142005,0.140734,0.286758,0.686271,0.044081,21.815789,402.605263,0.029148,0.048744,0.035570
3,Barcelona,609.973684,1.657895,0.242816,0.060930,0.047509,0.820782,0.096203,0.083015,0.260106,0.734803,0.026015,18.684211,569.763158,0.023055,0.046604,0.034088
4,Betis,460.657895,0.789474,0.212515,0.047674,0.030548,0.756161,0.120114,0.123725,0.444559,0.539960,0.041817,19.710526,415.552632,0.027913,0.043145,0.034664
5,Cádiz,328.842105,0.578947,0.178427,0.053026,0.047509,0.635867,0.148912,0.215221,0.240717,0.718150,0.061140,20.894737,271.631579,0.032194,0.037711,0.034911
6,Celta Vigo,488.763158,0.815789,0.227172,0.049568,0.043063,0.729128,0.130356,0.140516,0.339202,0.625101,0.045604,23.631579,426.052632,0.030218,0.049403,0.040758
7,Elche,391.105263,0.394737,0.192919,0.045204,0.036476,0.714736,0.137155,0.148109,0.414211,0.571794,0.043399,20.131579,344.842105,0.028324,0.041416,0.037629
8,Espanyol,393.710526,0.578947,0.214821,0.042816,0.030383,0.741414,0.128020,0.130566,0.307600,0.682374,0.042511,17.210526,353.631579,0.022890,0.036558,0.033923
9,Getafe,331.578947,0.368421,0.199588,0.047345,0.029724,0.613881,0.148563,0.237556,0.318810,0.616667,0.076190,22.052632,274.631579,0.026348,0.031124,0.034664


In [45]:

df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[3]}.csv")
print(df.columns)
def goal_shot_creation_preprocess(df, scale):
    '''Preprocess 'Goal and shot creation' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns
        - df: processed dataframe
        '''
    
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    # SCA, GCA already per-90'ed
    df.drop(columns = ['SCA', 'GCA'], inplace=True)

    # Dead ball things to drop
    df.drop(columns = ['PassDead', 'Fld', 'PassDead.1', 'Fld.1'], inplace=True)


    # executing the per-90 function on relevant stats
    df[["PassLive", "Drib", "Sh", "Def", "PassLive.1", "Drib.1", "Sh.1", "Def.1"]] = df[["PassLive",
     "Drib", "Sh", "Def", "PassLive.1", "Drib.1", "Sh.1", "Def.1"]].apply(lambda x: x/games)
    df.rename(columns ={"PassLive": "SCPass", "Drib": "SCDrib",  "Sh": "SCSh", "Def": "SCDef",
     "PassLive.1": "GCPass", "Drib.1": "GCDrib",  "Sh.1": "GCSh", "Def.1": "GCDef"}, inplace=True)
    
    # Drop columns we won't need anymore
    # df.drop(columns=['Att', 'Dead'], inplace=True) 
    
    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)
    
    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)
    
    return df 

Index(['Unnamed: 0', 'Squad', '# Pl', '90s', 'SCA', 'SCA90', 'PassLive',
       'PassDead', 'Drib', 'Sh', 'Fld', 'Def', 'GCA', 'GCA90', 'PassLive.1',
       'PassDead.1', 'Drib.1', 'Sh.1', 'Fld.1', 'Def.1'],
      dtype='object')


In [46]:
newer_df = goal_shot_creation_preprocess(df)
newer_df

,Squad,SCA90,SCPass,SCDrib,SCSh,SCDef,GCA90,GCPass,GCDrib,GCSh,GCDef
0,Alavés,14.11,8.684211,0.710526,0.947368,0.500000,1.26,0.605263,0.078947,0.131579,0.078947
1,Athletic Club,19.03,12.868421,1.000000,0.921053,0.631579,1.84,1.263158,0.131579,0.052632,0.078947
2,Atlético Madrid,18.76,13.578947,1.026316,0.842105,0.368421,2.66,2.026316,0.105263,0.184211,0.105263
3,Barcelona,22.42,17.473684,1.236842,0.736842,0.421053,2.97,2.289474,0.184211,0.210526,0.000000
4,Betis,21.97,15.973684,1.157895,0.894737,0.736842,2.50,1.710526,0.184211,0.131579,0.052632
5,Cádiz,15.68,10.447368,0.736842,1.078947,0.631579,1.42,0.842105,0.052632,0.131579,0.105263
6,Celta Vigo,16.13,12.473684,0.500000,0.894737,0.394737,1.84,1.394737,0.052632,0.236842,0.078947
7,Elche,13.71,9.973684,0.894737,0.473684,0.447368,1.53,1.078947,0.184211,0.052632,0.052632
8,Espanyol,16.92,12.578947,0.789474,0.763158,0.447368,1.68,1.131579,0.078947,0.184211,0.052632
9,Getafe,16.39,10.842105,0.789474,0.842105,0.657895,1.47,1.052632,0.078947,0.052632,0.078947


In [98]:

df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[4]}.csv")
print(df.columns)
def possession_preprocess(df, scale):
    '''Preprocess 'Possession' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns
        - df: processed dataframe
        '''
    
    # Drop these, they're useless
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    
    # Stats to drop (either taken into account, irrelevant or too small to matter)
    df.drop(columns = ["Touches", "Live", "Megs" ], inplace=True)

    # executing the per-90 function on relevant stats
    df[['Succ', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen', 'Att',
       '#Pl', 'Carries', 'TotDist', 'PrgDist', 'Prog', '1/3',
       'CPA', 'Mis', 'Dis', 'Targ', 'Rec', 'Prog.1']] = df[['Succ', 'Def Pen', 'Def 3rd',
      'Mid 3rd', 'Att 3rd', 'Att Pen', 'Att', '#Pl', 'Carries', 'TotDist', 'PrgDist',
       'Prog', '1/3','CPA', 'Mis', 'Dis', 'Targ', 'Rec', 'Prog.1']].apply(lambda x: x/games)


    # Touches per area don't add up to total touches stat. Add them up, then normalize so it's
    # proportion of actions at each location; Also think "Pen" touches are taken from that third, 
    # so subtract "Pen" touches from that area.
    # (Dead ball touches is included in here? Have to assume it won't change proportion too much, I guess) 

    total_touches = df['Def 3rd'] + df['Mid 3rd'] + df['Att 3rd']
    df["Def 3rd"] = df["Def 3rd"] - df["Def Pen"] 
    df["Att 3rd"] = df["Att 3rd"] - df["Att Pen"]
    df[['Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen']] =  df[['Def Pen', 'Def 3rd', 
    'Mid 3rd', 'Att 3rd', 'Att Pen']].apply(lambda x: x/total_touches)


    # Make % between 0-1
    df[["Poss", "Succ%", "Rec%"]] = df[["Poss", "Succ%", "Rec%"]].apply(lambda x: x/100)
    df.rename(columns={"Succ%": "DribSucc%"}, inplace=True)

    df['#Pl'] = df['#Pl']/df['Succ'] 
    df.rename(columns={'#Pl': '#Pl/Drib'},inplace=True)
    
    # Do some carries normalizing
    df['Prog'] = df['Prog']/df['Carries'] 
    df.rename(columns={'Prog': 'PropProg'},inplace=True)

    df['PrgDist'] = df['PrgDist']/df['TotDist'] 
    df.rename(columns={'PrgDist': 'PropPrgDist'},inplace=True)
    
    df['TotDist'] = df['TotDist']/df['Carries'] 
    df.rename(columns={'TotDist': 'TotDist/Carry'},inplace=True)

    df['1/3'] = df['1/3']/df['Carries'] 
    df.rename(columns={'1/3': 'Prop-1/3'},inplace=True)

    df['Mis'] = df['Mis']/df['Carries'] 
    df.rename(columns={'Mis': 'Mis/Carry'},inplace=True)
    
    df['Dis'] = df['Dis']/df['Carries'] 
    df.rename(columns={'Dis': 'Dis/Carry'},inplace=True)
    
    df['CPA'] = df['CPA']/df['Carries'] 
    df.rename(columns={'CPA': 'PropCPA'},inplace=True)

    # And some receiving normalizing
    
    df['Prog.1'] = df['Prog.1']/df['Rec'] 
    df.rename(columns={'Prog.1': 'Prog/Rec'},inplace=True)
    # Drop columns we won't need anymore
    df.drop(columns=['Succ', 'Rec'], inplace=True) 
    
    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)
    
    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)

    return df 

Index(['Unnamed: 0', 'Squad', '# Pl', 'Poss', '90s', 'Touches', 'Def Pen',
       'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen', 'Live', 'Succ', 'Att',
       'Succ%', '#Pl', 'Megs', 'Carries', 'TotDist', 'PrgDist', 'Prog', '1/3',
       'CPA', 'Mis', 'Dis', 'Targ', 'Rec', 'Rec%', 'Prog.1'],
      dtype='object')


In [99]:
poss_df = possession_preprocess(df)
poss_df

,Squad,Poss,Def Pen,Def 3rd,Mid 3rd,Att 3rd,Att Pen,Att,DribSucc%,#Pl/Drib,...,TotDist/Carry,PropPrgDist,PropProg,Prop-1/3,PropCPA,Mis/Carry,Dis/Carry,Targ,Rec%,Prog/Rec
0,Alavés,0.418,0.122664,0.200752,0.447213,0.195501,0.033870,13.500000,0.507,1.100000,...,4.768187,0.517553,0.096144,0.028309,0.010683,0.049354,0.035680,328.236842,0.789,0.085264
1,Athletic Club,0.481,0.098428,0.184871,0.452542,0.224383,0.039775,14.210526,0.504,1.113971,...,5.166959,0.540950,0.118073,0.037978,0.012341,0.037261,0.032325,407.315789,0.850,0.102342
2,Atlético Madrid,0.506,0.088285,0.208557,0.453604,0.209896,0.039657,15.078947,0.565,1.058642,...,4.928872,0.543374,0.109731,0.031724,0.009818,0.031056,0.025579,471.657895,0.854,0.081705
3,Barcelona,0.648,0.080320,0.171672,0.483307,0.225670,0.039031,17.894737,0.590,1.077307,...,4.943706,0.576966,0.118924,0.030517,0.012913,0.025439,0.021473,631.131579,0.903,0.077087
4,Betis,0.541,0.100586,0.202995,0.455431,0.203233,0.037755,16.631579,0.549,1.103746,...,4.898981,0.526527,0.104263,0.029780,0.011354,0.030299,0.026406,477.289474,0.871,0.078652
5,Cádiz,0.409,0.119248,0.222653,0.431927,0.190364,0.035809,12.605263,0.499,1.158996,...,5.571711,0.535988,0.106391,0.036184,0.009586,0.043609,0.036466,332.157895,0.818,0.082833
6,Celta Vigo,0.556,0.093854,0.208371,0.469231,0.197888,0.030656,13.394737,0.538,1.094891,...,5.260298,0.549894,0.111971,0.034487,0.008316,0.030942,0.025463,496.684211,0.858,0.071649
7,Elche,0.484,0.120281,0.226456,0.448748,0.179392,0.025122,14.921053,0.513,1.085911,...,4.862638,0.491892,0.093733,0.028290,0.005550,0.039698,0.031296,406.263158,0.849,0.071429
8,Espanyol,0.471,0.124188,0.220268,0.442966,0.184072,0.028506,15.500000,0.542,1.084639,...,5.198333,0.515849,0.101043,0.035733,0.009459,0.033556,0.031604,408.736842,0.865,0.069504
9,Getafe,0.412,0.108170,0.201761,0.466339,0.192296,0.031433,12.105263,0.489,1.115556,...,4.854322,0.492662,0.096990,0.031749,0.007453,0.047527,0.035040,337.157895,0.815,0.079916


In [ ]:
df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[6]}.csv")
print(df.columns)
def defense_preprocess(df, type):
    '''Preprocess 'Defensive actions' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns￼￼
        - df: processed dataframe
        '''
    
    # Drop these, they're useless
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    
    # Stats to drop (either taken into account, or too small to matter)
    df.drop(columns = [], inplace=True)

    # executing the per-90 function on relevant stats
    df[["Tkl", "TklW", "Def 3rd", "Mid 3rd", "Att 3rd",  "Att", "Press", "Def 3rd.1", "Mid 3rd.1", 
    "Att 3rd.1", "Blocks", "Sh", "Pass", "Int", "Clr"]] = df[["Tkl", "TklW", "Def 3rd", "Mid 3rd", "Att 3rd", 
    "Att", "Press", "Def 3rd.1", "Mid 3rd.1", "Att 3rd.1", "Blocks", "Sh", "Pass", "Int", "Clr"]].apply(lambda x: x/games)

    # Find proportion of actions at each location
    df['TklW'] = df['TklW']/df['Tkl']
    df['Def 3rd'] = df['Def 3rd']/df['Tkl']
    df['Mid 3rd'] = df['Mid 3rd']/df['Tkl']
    df['Att 3rd'] = df['Att 3rd']/df['Tkl']
    
    # Make between 0-1
    df[["Tkl%", "%"]] = df[["Tkl%", "%"]].apply(lambda x: x/100)
    
    df['Def 3rd.1'] = df['Def 3rd.1']/df["Press"]
    df['Mid 3rd.1'] = df['Mid 3rd.1']/df["Press"]
    df['Att 3rd.1'] = df['Att 3rd.1']/df["Press"]

    df["Sh"] = df["Sh"]/df["Blocks"]
    df["Pass"] = df["Pass"]/df["Blocks"]
    
    df.rename(columns ={"TklW": "Tkl%", "Def 3rd": "PropTkl Def 3rd", "Mid 3rd": "PropTkle Mid 3rd", "Att 3rd": "PropTkl Att 3rd",
     "Att": "DribTkl", "Tkl%": "DribTkl%", "%": "SuccPress%", "Def 3rd.1": "PropPress Def 3rd", "Mid 3rd.1": "PropPress Def 3rd", 
     "Att 3rd.1": "PropPress Def 3rd", "Sh": "ShBlock%", "Pass": "PassBlock%"}, inplace=True)
    
    
    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)
    
    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)
    
    # Drop columns we won't need anymore
    # df.drop(columns=['Tkl'], inplace=True) 
    
    return df 

In [113]:
df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[6]}.csv")
print(df.columns)
def playing_time_preprocess(df, scale):
    '''Preprocess 'Playing time' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns￼￼
        - df: processed dataframe
        '''
    
    # Drop these, they're useless
    df.drop(columns=['Unnamed: 0', '# Pl'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    # executing the per-90 function on relevant stats
    df[["onG", "onGA", "onxG", "onxGA"]] = df[["onG", "onGA", "onxG", "onxGA"]].apply(lambda x: x/games)
    
    # Find g-xg diff

    df["onG-xG"] = df['onG'] - df['onxG']
    df["onGA-xGA"] = df['onGA'] - df['onxGA']
    df["G-xG+/-"] = df['+/-'] - df['xG+/-']

    # Stats to drop (either taken into account, or too small to matter)
    df.drop(columns = ['Age', 'MP', 'Min', 'Mn/MP', 'Min%', 'Starts', 'Mn/Start', 'Compl',
    'Subs', 'Mn/Sub', 'unSub', 'PPM', '+/-', 'xG+/-'], inplace=True)

    # All stats are per-90 normalized, so remove that from label
    df.rename(columns={'+/-90': '+/-', 'xG+/-90': 'xG+/-'}, inplace=True)

    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)
    
    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)
    
    return df 

Index(['Unnamed: 0', 'Squad', '# Pl', 'Age', 'MP', 'Min', 'Mn/MP', 'Min%',
       '90s', 'Starts', 'Mn/Start', 'Compl', 'Subs', 'Mn/Sub', 'unSub', 'PPM',
       'onG', 'onGA', '+/-', '+/-90', 'onxG', 'onxGA', 'xG+/-', 'xG+/-90'],
      dtype='object')


In [114]:
pt_df = playing_time_preprocess(df)
pt_df

,Squad,onG,onGA,+/-,onxG,onxGA,xG+/-,onG-xG,onGA-xGA,G-xG+/-
0,Alavés,0.815789,1.710526,-0.89,0.971053,1.536842,-0.56,-0.155263,0.173684,-12.6
1,Athletic Club,1.131579,0.947368,0.18,1.318421,1.005263,0.32,-0.186842,-0.057895,-5.0
2,Atlético Madrid,1.710526,1.131579,0.58,1.389474,0.886842,0.50,0.321053,0.244737,2.9
3,Barcelona,1.789474,1.000000,0.79,1.634211,0.947368,0.69,0.155263,0.052632,3.9
4,Betis,1.631579,1.052632,0.58,1.452632,1.165789,0.29,0.178947,-0.113158,11.1
5,Cádiz,0.921053,1.342105,-0.42,1.089474,1.389474,-0.30,-0.168421,-0.047368,-4.6
6,Celta Vigo,1.131579,1.131579,0.00,1.152632,1.226316,-0.07,-0.021053,-0.094737,2.8
7,Elche,1.052632,1.368421,-0.32,0.915789,1.605263,-0.69,0.136842,-0.236842,14.2
8,Espanyol,1.052632,1.394737,-0.34,0.994737,1.371053,-0.38,0.057895,0.023684,1.3
9,Getafe,0.868421,1.078947,-0.21,0.776316,1.028947,-0.25,0.092105,0.050000,1.6


In [12]:

df = pd.read_csv(f"../data/raw/team/{leagues[0]}-2021-2022-team-{stats[7]}.csv")
print(df.columns)
def misc_preprocess(df, scale):
    '''Preprocess 'Miscallaneous' stat from FBRef
    Parameters:
        - df: dataframe of Statsbomb statistical data taken from FBRef 
    Returns￼￼
        - df: processed dataframe
        '''
    
    # Drop these, they're useless
    df.drop(columns=['Unnamed: 0'], inplace=True)

    # Function to divide relevant columns by games played
    games = df['90s'][0]
    df.drop(columns=['90s'], inplace=True)
    
    # Stats to drop (either taken into account, or too small to matter)
    # OG could be interesting, but there are so few that it could skew results
    df.drop(columns = ['2CrdY', 'Crs', 'Int', 'TklW', 'OG'], inplace=True)
    
    # executing the per-90 function on relevant stats
    df[["CrdY", "CrdR", "Fls", "Fld", "Off", "PKwon", "PKcon", "Recov", "Won", "Lost",
    ]] = df[["CrdY", "CrdR", "Fls", "Fld", "Off", "PKwon", "PKcon", "Recov", "Won", 
    "Lost"]].apply(lambda x: x/games)
    
    # Turn % into number between 0-1
    df[["Won%"]] = df[["Won%"]].apply(lambda x: x/100)
    df.rename(columns={"Won%": "DuelWin%"}, inplace=True)

    # Find total number of aerial duels
    df['AerialDuels'] = df['Won'] + df['Lost'] 
    

    # Stats to drop (either taken into account, or too small to matter)
    df.drop(columns = ['Won', 'Lost'], inplace=True)

    # All stats are per-90 normalized, so remove that from label
    df.rename(columns={'+/-90': '+/-', 'xG+/-90': 'xG+/-'}, inplace=True)

    if scale == 'Team': 
        df.drop(columns=['# Pl'], inplace=True)

    if scale == 'Individual':
        df.drop(columns=['Matches', 'Age'], inplace=True)
    
    return df 

Index(['Unnamed: 0', 'Squad', '# Pl', '90s', 'CrdY', 'CrdR', '2CrdY', 'Fls',
       'Fld', 'Off', 'Crs', 'Int', 'TklW', 'PKwon', 'PKcon', 'OG', 'Recov',
       'Won', 'Lost', 'Won%'],
      dtype='object')


In [13]:
misc_df = misc_preprocess(df)
misc_df

,Squad,CrdY,CrdR,Fls,Fld,Off,PKwon,PKcon,Recov,DuelWin%,AerialDuels
0,Alavés,2.447368,0.105263,16.131579,13.500000,1.894737,0.157895,0.236842,88.605263,0.559,48.921053
1,Athletic Club,2.368421,0.105263,14.921053,14.894737,2.315789,0.131579,0.078947,89.210526,0.498,36.815789
2,Atlético Madrid,2.921053,0.184211,14.052632,16.052632,2.236842,0.105263,0.210526,84.552632,0.512,34.157895
3,Barcelona,2.552632,0.131579,14.605263,16.421053,2.605263,0.157895,0.078947,79.631579,0.524,26.657895
4,Betis,2.289474,0.157895,14.421053,14.842105,1.868421,0.105263,0.078947,80.789474,0.489,32.315789
5,Cádiz,2.763158,0.078947,15.157895,13.315789,1.815789,0.131579,0.315789,84.763158,0.476,39.473684
6,Celta Vigo,2.289474,0.078947,15.157895,14.342105,2.184211,0.078947,0.184211,89.973684,0.468,40.052632
7,Elche,2.736842,0.157895,15.605263,13.947368,1.947368,0.026316,0.263158,76.447368,0.478,31.447368
8,Espanyol,2.605263,0.184211,13.578947,15.789474,1.815789,0.078947,0.131579,72.868421,0.447,31.815789
9,Getafe,3.184211,0.210526,16.605263,14.078947,1.394737,0.026316,0.105263,83.789474,0.495,46.289474


In [ ]:
scales = ['team', 'individual']
stats = ['shooting', 'passing', 'passing_types', 'gca', 'possession', 'defense', 'playingtime', 'misc']
leagues = ['La-Liga', 'Premier-League', 'Ligue-1', 'Bundesliga',  'Serie-A', 'Major-League-Soccer']

In [ ]:
# Loop through above lists to get all data. Also will need to append dfs -> stats along axis 1, leagues along axis 0

for scale in scales:
    scale_frames = []
    for league in leagues: 
        league_frames = []
        df = pd.read_csv(f"../data/raw/{scale}/{leagues[0]}-2021-2022-team-{stats[0]}.csv")
        df = shooting_preprocess(df, scale)
        league_frames.append(df)
        df  = passing_preprocess(df, scale)
        league_frames.append(df)
        df  = pass_types_preprocess(df, scale)
        league_frames.append(df)
        df  = goal_shot_creation_preprocess(df, scale)
        league_frames.append(df)
        df  = possession_preprocess(df, scale)
        league_frames.append(df)
        df  = defense_preprocess(df, scale)
        league_frames.append(df)
        df  = playing_time_preprocess(df, scale)
        league_frames.append(df)
        df  = misc_preprocess(df, scale)
        league_frames.append(df)
            
                # Need to change functions to account for individual
        #append to total df